# API DATA WRANGLING

This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store

In [1]:
ls


 Volume in drive C is Windows
 Volume Serial Number is B403-50FE

 Directory of C:\Users\SANJAY\0projects\02Springboard\API_Data_Wrangling_Mini_Project

10-07-2020  04:27 PM    <DIR>          .
10-07-2020  04:27 PM    <DIR>          ..
10-07-2020  02:33 PM    <DIR>          .ipynb_checkpoints
10-07-2020  04:27 PM            12,854 api_data_wrangling_mini_project.ipynb
               1 File(s)         12,854 bytes
               3 Dir(s)  127,927,115,776 bytes free


In [2]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = open("C:/Users/SANJAY/Desktop/Springboard-Notes/api_key.txt")
API_KEY=API_KEY.read()


Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [3]:
# First, import the relevant modules
import requests
import json
import numpy as np
import quandl

In [4]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?limit=1&api_key='+ API_KEY
r = requests.get(url)

In [5]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(r.json())

{'dataset_data': {'limit': 1, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2000-06-07', 'end_date': '2020-07-09', 'frequency': 'daily', 'data': [['2020-07-09', 91.15, 91.35, 89.45, 89.55, None, 181776.0, 16405326.65, None, None, None]], 'collapse': None, 'order': None}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

### 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [6]:
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key=' + API_KEY
r  = requests.get(url)
r

<Response [200]>

In [7]:
#convert to json format and check  type
type(r.json())

dict

### 2. Convert the returned JSON object into a Python dictionary.


In [8]:
json_df = dict(r.json())
print(type(json_df))
json_df['dataset_data']['data'][:1]

<class 'dict'>


[['2017-12-29',
  51.76,
  51.94,
  51.45,
  51.76,
  None,
  34640.0,
  1792304.0,
  None,
  None,
  None]]

### 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [9]:
print(json_df['dataset_data']['column_names'])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [10]:
opening_price = [x[1] for x in json_df['dataset_data']['data'] 
                if x[1] is not None]
print('Highest Opening price for the stock:',max(opening_price))
print('Lowest Opening price for the stock:',min(opening_price))

Highest Opening price for the stock: 53.11
Lowest Opening price for the stock: 34.0


### 4. What was the largest change in any one day (based on High and Low price)?

In [11]:
price_change = [x[2]-x[3] for x in json_df['dataset_data']['data']]
print('largest chane in any one data based on high and Low Price:',
     float(max(price_change))) #type to int

largest chane in any one data based on high and Low Price: 2.8100000000000023


### 5. What was the largest change between any two days (based on Closing Price)?

In [12]:
#Since close is 5th value in the dataset
close = [x[4] for x in json_df['dataset_data']['data']]
two_day=[]
for i in range(len(close)):
    if (i+2) < len(close):
        two_day.append(abs(close[i]-close[i+2]))
print('Largest change b/w two days:',float(max(two_day)))

Largest change b/w two days: 3.1499999999999986


### 6. What was the average daily trading volume during this year?

In [13]:
trading_volume = [x[6] for x in json_df['dataset_data']['data']]
print('Avg trading volume:',np.average(trading_volume))

Avg trading volume: 89124.33725490196


### 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [14]:
print('Median trading volume:',np.median(trading_volume))

Median trading volume: 76286.0
